In [ ]:
from wholeslidedata.annotation.wholeslideannotation import WholeSlideAnnotation
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata.annotation.types import PolygonAnnotation as Polygon
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import numpy as np
import os
from tqdm import tqdm

import cv2

from py.helpers import get_outlines, get_area, get_patch, get_sub_areas, patch_empty, concat_one, BARRET_ROOT
import os

os.add_dll_directory(r'C:\Program Files\openslide-win64\bin') # for openslide

LANS_DIR = os.path.join(BARRET_ROOT, 'LANS_001-923')
LANS_BIOP_ROOT = os.path.join(BARRET_ROOT, 'p53_experiment_luuk_biopsy-level_no-HE')
LANS_BIOP_DIR = os.path.join(LANS_BIOP_ROOT, 'P53_score_high_consensus')

In [ ]:
def get_edge_pixels(img):
    edge_pixels = np.array([np.array([i, 0]) for i in range(img.shape[0])] + \
                       [np.array([img.shape[0]-1, i]) for i in range(img.shape[1]-1, 0, -1)] + \
                       [np.array([i, img.shape[1]-1]) for i in range(img.shape[0]-1, 0, -1)] + \
                       [np.array([0, i]) for i in range(img.shape[1]-1, 0, -1)]).astype(np.uint16)
    return edge_pixels

In [ ]:
def get_most_common_mask(image, similarity=30, exclude=None, only_edge=False):
    """Get a mask of the most common color in an image"""
    # Flatten the image to a 2D array of pixels and 3 color values (RGB)
    flat = image.reshape(-1, 3)
    if only_edge:
        edge_pixels = get_edge_pixels(image)
        image_edge = image[edge_pixels[:, 0], edge_pixels[:, 1]]
        flat = image_edge.reshape(-1, 3)

    if exclude is not None:
        # Exclude anything close to the excluded color
        flat = flat[np.linalg.norm(flat.astype(np.float32) - exclude.astype(np.float32), axis=-1) > similarity]

    unique, counts = np.unique(flat, return_counts=True, axis=0)

    # Display the top 10 unique colors and their counts
    # Sort in descending order by count
    sorted_indices = np.argsort(-counts)
    counts = counts[sorted_indices]
    unique = unique[sorted_indices]

    most_common = unique[0]

    print(f'Most common color: {most_common}, count: {counts[0]}')

    # Make a mask of every pixel that is close to the most common color
    mask = np.linalg.norm(image.astype(np.float32) - most_common.astype(np.float32), axis=-1) > similarity

    return mask

def get_interpolations(p1s, p2s, factor):
    """get point in between two points, given a factor between 0 and 1 (0.5 is halfway). p1s and p2s are arrays of points"""
    return p1s + (p2s - p1s) * factor

def sample_points_within_contour(contour, factor=0.5):
    """sample n points within a contour"""
    center = contour.mean(axis=0)
    # For each point in the contour, get an interpolation between the center and the point
    # The interpolation factor is 0.5, so the interpolation is halfway between the center and the point
    # This will give us a set of points that are within the contour
    interpolations = get_interpolations(np.stack([center]*len(contour), axis=0), contour, factor)
    return interpolations.astype(np.int32)

def get_contour_inner(mask, contour, factor=0.9, validate=True):
    """Return 1 if the contour is around a white area, 0 if it is around a black area"""
    # Get the points within the contour
    points = sample_points_within_contour(contour, factor)
    # Filter out points that are outside the contour
    if validate:
        mask_points = [cv2.pointPolygonTest(contour.astype(np.float32), tuple(p.astype(np.float32)), False) >= 0 for p in points]
        points = points[mask_points]
    # Get the values of the mask at those points
    values = mask[points[:, 1], points[:, 0]]
    # Get the mean value
    if mask.max() == 255:
        return values.mean() > 127
    else:
        return values.mean() > 0.5  

def split_contour(c, size_threshold, distance_threshold=30, skip_if_large=1e5):
    """Split a contour into two contours on a thin bottleneck, if the two resulting contours are both larger than the size threshold"""
    # Return the contour if it is smaller than double the size threshold
    area = cv2.contourArea(c)
    if area < size_threshold * 2:
        return [c]
    elif area > skip_if_large:
        return [c]

    # Get the points of the contour
    points = c[:, 0, :]

    # Create pairwise distances between all points
    distances = np.linalg.norm(points[:, None, :] - points[None, :, :], axis=-1)

    # We want to find a thin bottleneck in the contour, so we want to find pairs that are close together
    close_pairs = np.argwhere((distances < distance_threshold) & (distances > 0))

    # Get the unique pairs
    unique_pairs = np.unique(np.sort(close_pairs, axis=-1), axis=0)

    # Sort the pairs by distance
    unique_pairs = unique_pairs[np.argsort(distances[unique_pairs[:, 0], unique_pairs[:, 1]])]

    # For each pair, get the contour points between the two points
    # If the resulting contours are larger than the size threshold, return the two resulting contours
    # Otherwise, return the original contour

    # Get the points between the two points
    def get_points_between(p1, p2):
        # Get the indices of the points between the two points
        indices = np.arange(p1, p2)
        # Get the points between the two points
        return points[indices]
    
    # Get the two resulting contours
    def get_resulting_contours(p1, p2):
        # Get the points between the two points
        points_between = get_points_between(p1, p2)
        # Get the contour between the two points
        contour_between = points_between[:, None, :]
        # Get the contour outside the two points
        contour_outside = np.concatenate([points[:p1, None, :], points[p2:, None, :]], axis=0)
        return contour_between, contour_outside
    
    # Get the two resulting areas
    def get_resulting_areas(p1, p2):
        contour_between, contour_outside = get_resulting_contours(p1, p2)
        return cv2.contourArea(contour_between), cv2.contourArea(contour_outside)
    
    # Check for each pair if the resulting contours are larger than the size threshold
    # If they are, return the two resulting contours
    # Otherwise, return the original contour
    for p1, p2 in unique_pairs:
        area_between, area_outside = get_resulting_areas(p1, p2)
        if area_between < size_threshold or area_outside < size_threshold:
            continue
        # If the center of one is inside the other, we don't want to split it
        # Get the center of the two areas
        contour_between, contour_outside = get_resulting_contours(p1, p2)
        center_between = contour_between.mean(axis=0)
        center_outside = contour_outside.mean(axis=0)
        # Check if the center of one is inside the other
        if cv2.pointPolygonTest(contour_between, tuple(center_outside[0]), False) >= 0:
            continue
        if cv2.pointPolygonTest(contour_outside, tuple(center_between[0]), False) >= 0:
            continue 
        return get_resulting_contours(p1, p2)
    return [c]

In [ ]:
# Convert mask to grayscale
mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

# Get the contours of mask
contours, _ = cv2.findContours(mask_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Returns tuple (contours, hierarchy), where contours is a tuple of contours, 

# Get the contour with the largest area
contour = max(contours, key=cv2.contourArea)[:, 0, :]
# where each contour is an array of points of shape (n, 1, 2), where 1 is the number of contours and 2 is the number of points in each contour
center = contour.mean(axis=0)

# Sample points within the contour
points = sample_points_within_contour(contour, 0.99)

# Filter out points that are not within the contour
mask_points = [cv2.pointPolygonTest(contour.astype(np.float32), tuple(p.astype(np.float32)), False) >= 0 for p in points]
points = points[mask_points]

# Get mean color of each point
colors = mask[points[:, 1], points[:, 0]]
mean = colors.mean()
print(mean)

# Plot the points
plt.figure(figsize=(10, 10))
plt.imshow(mask)
plt.scatter(contour[:, 0], contour[:, 1], s=1, c='g')
plt.scatter(points[:, 0], points[:, 1], s=1, c='r')
# Put circle at the center
plt.scatter(center[0], center[1], s=10, c='b')
plt.show()



In [ ]:
spacing = 64.0 # microns per pixel

# Get first .tiff file in directory
files = os.listdir(LANS_BIOP_DIR)
files = [f for f in files if f.endswith('.tiff')]

for file_nr, file in enumerate(files[:]):
    file_path = os.path.join(LANS_BIOP_DIR, file)

    # Read the wsi
    try:
        wsi = WholeSlideImage(file_path)
        print(f"{file_nr}/{len(files)}: {file_path}")
    except:
        print(f"Could not read {file_path}")
        continue
    image = wsi.get_slide(spacing=spacing)

    # Null background is usually 254,254,254 or 254,0,0
    # Scanned background is usually around 225,225,225, we want to keep this, so we exclude it from the mask
    mask = get_most_common_mask(image, similarity=30, exclude=np.array([225, 225, 225]), only_edge=True)

    # Make a copy of the image as a UMat object
    image_um = cv2.UMat(image)

    # Get contours of the mask
    contours, hierarchy = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out the small contours
    contours = [c for c in contours if cv2.contourArea(c) > 1000]

    # Get bounding boxes for the contours
    boxes = [cv2.boundingRect(c) for c in contours]

    # Filter out boxes that have a much larger area than the contour (the contour is not close to a rectangle)
    boxes = [box for box, contour in zip(boxes, contours) if cv2.contourArea(contour) / (box[2] * box[3]) > 0.9]

    # Draw purple cross on the image at the center of each bounding box
    for x, y, w, h in boxes:
        cv2.line(image_um, (x + w // 2 - 10, y + h // 2), (x + w // 2 + 10, y + h // 2), (255, 0, 255), 2)
        cv2.line(image_um, (x + w // 2, y + h // 2 - 10), (x + w // 2, y + h // 2 + 10), (255, 0, 255), 2)

    # Draw bounding boxes on the image
    for x, y, w, h in boxes:
        cv2.rectangle(image_um, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the mask on top of the original image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(mask, cmap="gray")
    plt.subplot(1, 2, 2)
    plt.imshow(image_um.get())
    plt.show()

    # Adjust boxes for spacing
    boxes = [(x, y, w, h) for x, y, w, h in boxes]
    boxes = [(x * spacing*4, y * spacing*4, w*spacing, h*spacing) for x, y, w, h in boxes]

    # Take 2% off each side of the box
    boxes = [(x + w * 0.02, y + h * 0.02, w * 0.96, h * 0.96) for x, y, w, h in boxes]

    # Get patch of each box from the wsi
    new_spacing = 8.0
    patches = [wsi.get_patch(x, y, w/new_spacing, h/new_spacing, spacing=new_spacing, center=False) for x, y, w, h in boxes]

    # Display the patches in a horizontal row
    plt.figure(figsize=(5*len(patches), 5))
    for i, patch in enumerate(patches):
        mask = get_most_common_mask(patch, similarity=10)
        patch = patch.astype(np.uint8)
        show_img = patch.copy()

        # Opening and closing without changing to UMat
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel, iterations=1)

        # Make contours
        # A contour is a 3D array of points, these represent the points on the contour. 
        # The first dimension is the index of the point, the second is the index of the contour, and the third is the x and y coordinates of the point.
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Make mask 3 channels 0-255
        show_mask = np.stack([mask, mask, mask], axis=-1).astype(np.uint8) * 255

        # Filter out contours that are dark, or small
        size_threshold = (8/new_spacing)**2 * 7500
        new_contours = []
        for c in contours:
            if cv2.contourArea(c) < size_threshold:
                cv2.drawContours(show_mask, [c], 0, (255, 0, 0), 5)
                continue
            # If area of bounding box is almost the area of the patch
            x, y, w, h = cv2.boundingRect(c)
            if w * h > 0.9 * patch.shape[0] * patch.shape[1]:
                cv2.drawContours(show_mask, [c], 0, (255, 0, 255), 5)
                continue
            # If contour hits the edge of the patch
            edge_hits = [p for p in c[:, 0, :] if p[0] == 0 or p[0] == mask.shape[1]-1 or p[1] == 0 or p[1] == mask.shape[0]-1]
            for hit in edge_hits:
                # Draw a blue dot on the image at the edge hit
                cv2.circle(show_mask, tuple(hit), 50, (255, 255, 0), 50)
            edge_hits = len(edge_hits)
            if edge_hits > 2:
                cv2.drawContours(show_mask, [c], 0, (255, 255, 0), 5)
                cv2.putText(show_mask, f"hits: {edge_hits}", tuple(c[0, 0, :]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 5)
                continue
            if not get_contour_inner(mask, c[:,0,:], factor=0.9):
                cv2.drawContours(show_mask, [c], 0, (0, 255, 0), 5)
                cv2.putText(show_mask, "outer", tuple(c[0, 0, :]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 5)
                continue

            # Draw contour on the image
            cv2.drawContours(show_img, [c], 0, (0, 0, 0), 5)
            new_contours.append(c)
        contours = new_contours

        # Get bounding boxes for the contours
        boxes = [cv2.boundingRect(c) for c in contours]

        # Filter out boxes that have their center in another bounding box
        def box_center_in_box(box1, box2):
            x1, y1, w1, h1 = box1
            x2, y2, w2, h2 = box2
            return x1 + w1 // 2 > x2 and x1 + w1 // 2 < x2 + w2 and y1 + h1 // 2 > y2 and y1 + h1 // 2 < y2 + h2
        
        box_indices = [j for j, box1 in enumerate(boxes) if not any([box_center_in_box(box1, box2) for box2 in boxes if box1 != box2])]
        boxes = [boxes[j] for j in box_indices]
        contours = [contours[j] for j in box_indices]

        new_contours = []
        for c in contours:
            new_contours.extend(split_contour(c, size_threshold*3, distance_threshold=30, skip_if_large=2e5))


        # Draw contours and bounding boxes on the image
        for j in range(len(new_contours)):
            contour = new_contours[j]
            x, y, w, h = cv2.boundingRect(contour)
            center = (x + w // 2, y + h // 2)

            # Draw blue contour on the image
            contour_color = (0, 0, 255)
            cv2.drawContours(show_img, [contour], 0, contour_color, 5)
            # Draw green bounding box on the image
            cv2.rectangle(show_img, (x, y), (x + w, y + h), (0, 255, 0), 5)


            # Show the area of the contour in the top left corner
            area = cv2.contourArea(contour)
            cv2.putText(show_img, f"{area:.0f}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 5)

            # If the area is too large, draw large red cross through the bounding box
            if area > 2e5:
                cv2.line(show_img, (x, y), (x + w, y + h), (255, 0, 0), 5)
                cv2.line(show_img, (x + w, y), (x, y + h), (255, 0, 0), 5)
            else:
                # Draw large purple cross on the image at the center of each bounding box
                cv2.line(show_img, (x + w // 2 - 10, y + h // 2), (x + w // 2 + 10, y + h // 2), (0, 255, 0), 5)
                cv2.line(show_img, (x + w // 2, y + h // 2 - 10), (x + w // 2, y + h // 2 + 10), (0, 255, 0), 5)

        plt.subplot(2, len(patches), i + 1)
        plt.imshow(show_mask)
        plt.subplot(2, len(patches), i + 1 + len(patches))
        plt.imshow(show_img)

    # Make one legend for all axes, outside last plot
    plt.subplot(2, len(patches), i+1)
    plt.legend(handles=[
        plt.Rectangle((0, 0), 1, 1, color=(0, 0, 0)),
        mpatches.Circle((0, 0), 0.25, edgecolor=(1, 0, 0), facecolor="none", linewidth=2),
        mpatches.Circle((0, 0), 0.25, edgecolor=(1, 0, 1), facecolor="none", linewidth=2),
        mpatches.Circle((0, 0), 0.25, edgecolor=(1, 1, 0), facecolor="none", linewidth=2),
        mpatches.Circle((0, 0), 0.25, edgecolor=(0, 1, 0), facecolor="none", linewidth=2)], labels=[
            'Background',
            'Too small', 
            'Stretches across entire patch', 
            'Edge hit',
            'Surrounds background area',
            ], loc='upper left', bbox_to_anchor=(1, 1))

    # Make one legend for all axes, outside last plot
    plt.subplot(2, len(patches), i+1+len(patches))
    plt.legend(handles=[
        mpatches.Circle((0.5, 0.5), 0.25, edgecolor=(0,0,0), facecolor="none", linewidth=2), 
        mpatches.Circle((0.5, 0.5), 0.25, edgecolor=(0,0,1), facecolor="none", linewidth=2), 
        mpatches.Circle((0.5, 0.5), 0.25, edgecolor=(0,1,0), facecolor="none", linewidth=2), 
        mlines.Line2D([], [], color=(0,1,0), marker='+', linestyle='None', markersize=10, markeredgewidth=1),
        mlines.Line2D([], [], color=(1,0,0), marker='X', linestyle='None', markersize=10, markeredgewidth=1)
    ], labels=[
        'Inside another box',
        'Detected tissue',
        'Detected biopsy',
        'Center of biopsy',
        'Large area'
    ], loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.show()
